In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import subprocess
import os.path
import time
import math
import csv
#%matplotlib inline
    
from sklearn.datasets import make_regression, make_classification, load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical,np_utils
from keras.callbacks import EarlyStopping

# Some signal processing stuff
from scipy import signal
from scipy.io import wavfile

# IPython.display lets you put interactive widgets that can play audio and video, and display images
from IPython.display import Audio, Image

In [23]:
#Function from Alex Lin 
def bash(string):
    '''run a bash command and return stdout lines as a py list'''
    stdout=(subprocess.Popen(string, shell=True, stdout=subprocess.PIPE).stdout.read())
    output = str(stdout)[2:-1].split('\\n')[:-1]
    #output = str(stdout).split('\\n')[1:-1]
    #output[0] = output[0][2:]
    return output

# Get all the spectrograms in the Data Folder
# then put in a numpy array
# Get labels too

In [24]:
def load_process_data(data_folder, divisor):
    lis = bash('ls '+ data_folder)

    img_array = []
    labels = []

    for path in lis:
        path_ = data_folder+ path
        if path_ == '../Data/06_mp3-314.png':
            img = cv2.imread(path_,1)
            img_norm = img/255
            img_norm.resize(int(513/divisor),int(800/divisor),3)
            img_array.append(img_norm)
            labels.append(path[0:6])
            break;

        img = cv2.imread(path_,1)
        img_norm = img/255
        img_norm.resize(int(513/divisor),int(800/divisor),3)
        img_array.append(img_norm)
        labels.append(path[0:6])
    
    img_array_nd = np.array(img_array)
    
    le = LabelEncoder()
    y_cat = le.fit_transform(labels)
    y_cat = y_cat.reshape(-1,1)
    
    ohe = OneHotEncoder()
    y_cat = ohe.fit_transform(y_cat).toarray()
    
    X = img_array_nd
    y = y_cat
    
    return X,y,labels

In [25]:
# def load_process_data_batches(data_folder, divisor):
#     lis = bash('ls '+ data_folder)
    
#     img_array_nd = np.array([])
#     img_array = []
#     labels = []
#     print(len(lis)//1000)
#     for i in range(len(lis)//1000):
#         img_array = []
#         for path in lis[i*1000 : (i+1)*1000]:
            
#             path_ = data_folder+ path
#         #     if path == 'E:/spectrogram/06_mp3/06_mp3-314.png':
#         #         img = cv2.imread(path,1)
#         #         img_norm = img/255
#         #         img_array.append(img)
#         #         labels.append(path[15:21])
#         #         break;

#             img = cv2.imread(path_,1)
#             img_norm = img/255
#             img_norm.resize(int(513/divisor),int(800/divisor),3)
#             img_array.append(img_norm)
#             labels.append(path[0:6])
#         img_array_nd = np.append(img_array_nd,np.array(img_array))
    
#     le = LabelEncoder()
#     y_cat = le.fit_transform(labels)
#     y_cat = y_cat.reshape(-1,1)
    
#     ohe = OneHotEncoder()
#     y_cat = ohe.fit_transform(y_cat).toarray()
    
#     X = img_array_nd
#     y = y_cat
    
#     return X,y,labels

In [26]:
X,y,labels = load_process_data("../Data/",2)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1,stratify = y)

In [27]:
X.shape

(2508, 256, 400, 3)

In [33]:
model2 = Sequential()

model2.add(Conv2D(filters = 8,                   # number of filters
                         kernel_size = 5,        # height/width of filter
                         padding = "same",
                         activation='relu',      # activation function 
                         input_shape=(X_test[0].shape))) # shape of input (dimensions of training image)

model2.add(Conv2D(16, kernel_size = 5, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))
 
model2.add(Conv2D(16, kernel_size = 3, activation='relu'))
model2.add(Conv2D(16, kernel_size = 3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Flatten())
model2.add(Activation('relu'))
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.5))
model2.add(Dense(6, activation='softmax'))

In [34]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 256, 400, 8)       608       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 252, 396, 16)      3216      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 126, 198, 16)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 126, 198, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 124, 196, 16)      2320      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 122, 194, 16)      2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 61, 97, 16)        0         
__________

In [35]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=3,verbose=1, mode='auto')
callbacks_list = [earlystop]

In [37]:
model2.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test,y_test),callbacks=callbacks_list)

Train on 2257 samples, validate on 251 samples
Epoch 1/100
2257/2257 [==============================] - 47s 21ms/step - loss: 1.8451 - acc: 0.3079 - val_loss: 1.1166 - val_acc: 0.5697
Epoch 2/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.8536 - acc: 0.6779 - val_loss: 0.7723 - val_acc: 0.7450
Epoch 3/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.4323 - acc: 0.8463 - val_loss: 0.6049 - val_acc: 0.7968
Epoch 4/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.2351 - acc: 0.9211 - val_loss: 0.3890 - val_acc: 0.8526
Epoch 5/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.1364 - acc: 0.9557 - val_loss: 0.2087 - val_acc: 0.9363
Epoch 6/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.1133 - acc: 0.9623 - val_loss: 0.7647 - val_acc: 0.7769
Epoch 7/100
2257/2257 [==============================] - 22s 10ms/step - loss: 0.1063 - acc: 0.9681 - val_loss: 0.2586 - val_acc: 0

In [38]:
model2.save('model_03_6ppl.h5')